In [25]:
import sys
sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')
sys.path.append("../src")

In [26]:
import json

from utils import data_utils
from utils import conceptset_utils

In [3]:
from pathlib import Path
root = Path.cwd().parent.parent

import os
os.chdir(root)

In [5]:
"""
CLASS_SIM_CUTOFF: Concenpts with cos similarity higher than this to any class will be removed
OTHER_SIM_CUTOFF: Concenpts with cos similarity higher than this to another concept will be removed
MAX_LEN: max number of characters in a concept

PRINT_PROB: what percentage of filtered concepts will be printed
"""

CLASS_SIM_CUTOFF = 0.85
OTHER_SIM_CUTOFF = 0.9
MAX_LEN = 30
PRINT_PROB = 1

dataset = "doctor_nurse"
device = "cuda"

save_name = "data/concept_sets/{}_filtered_new.txt".format(dataset)

In [17]:
#EDIT these to use the initial concept sets you want

with open(root / "data/concept_sets/gpt3_init/gpt3_{}_important_new.json".format(dataset), "r") as f:
    important_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_superclass_new.json".format(dataset), "r") as f:
    superclass_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_around_new.json".format(dataset), "r") as f:
    around_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_job_new.json".format(dataset), "r") as f:
    job_dict = json.load(f)
    
with open(root / data_utils.LABEL_FILES[dataset], "r") as f:
    classes = f.read().split("\n")

In [20]:
concepts = set()

for values in important_dict.values():
    concepts.update(set(values))

for values in superclass_dict.values():
    concepts.update(set(values))
    
for values in around_dict.values():
    concepts.update(set(values))
    
for values in job_dict.values():
    concepts.update(set(values))

print(len(concepts))

43


In [21]:
concepts = conceptset_utils.remove_too_long(concepts, MAX_LEN, PRINT_PROB)

43 43


In [22]:
concepts = conceptset_utils.filter_too_similar_to_cls(concepts, classes, CLASS_SIM_CUTOFF, device, PRINT_PROB)

43
Class:doctor - Deleting a doctor
Class:nurse - Deleting a nurse
41
Class:doctor - Concept:medical professional, sim:0.871 - Deleting medical professional

40


In [23]:
concepts = conceptset_utils.filter_too_similar(concepts, OTHER_SIM_CUTOFF, device, PRINT_PROB)

health care worker - healthcare worker , sim:0.9728 - Deleting healthcare worker
39


In [24]:
with open(save_name, "w") as f:
    f.write(concepts[0])
    for concept in concepts[1:]:
        f.write("\n" + concept)